In [2]:
from tinyrpc.protocols.jsonrpc import JSONRPCProtocol
from tinyrpc.transports.http import HttpPostClientTransport
from tinyrpc import RPCClient

rpc_client = RPCClient(
    JSONRPCProtocol(),
    HttpPostClientTransport('http://127.0.0.1:8999/'))

remote_server = rpc_client.get_proxy()

# call a method
result = remote_server.blockNumber()

print("Server answered:", result)

Server answered: 4304


In [3]:
remote_server.blockNumber()

4304

In [12]:
remote_server.systemScript()

{'name': 'system',
 'elf_path': 'system',
 'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8',
 'hash_type': 'type',
 'tx_hash': '0x62b47d45ee689fb852af9002c318dcea1aeb9a4632991429fc3ec38676e356e3',
 'index': '0x0',
 'dep_type': 'dep_group'}

In [9]:
import sqlite3
cx = sqlite3.connect("/home/rink/work/github/ckb-rpc-wrapper/sync.db")
cu = cx.cursor()
cu.execute("select * from info")
print(cu.fetchall())

[(4293,)]


In [11]:
cu.execute("SELECT COUNT(*) from livecells")
print(cu.fetchall())

[(4281,)]


In [2]:
import z3;
print(z3.get_version_string())

4.8.6


In [4]:
x = z3.Int('x')
y = z3.Int('y')
z3.solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


In [5]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('factorial, N')
 

factorial[N] = N*factorial[N-1] 

factorial[1] = 1
 
print(factorial[3]==N)

N
-
6


In [13]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////home/rink/work/github/ckb-rpc-wrapper/sync.db?check_same_thread=False', echo=True)
print(engine)

Engine(sqlite:////home/rink/work/github/ckb-rpc-wrapper/sync.db?check_same_thread=False)


In [14]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
print(Base)

<class 'sqlalchemy.ext.declarative.api.Base'>


In [15]:
from sqlalchemy import Column, Integer, String, Text

# 定义映射类Info，其继承上一步创建的Base
class Info(Base):
    # 指定本类映射到info表
    __tablename__ = 'info'
    
    tip_height = Column(Integer, primary_key=True)

    def __repr__(self):
        return "<Info(tip_height='%s')>" % (
                   self.tip_height)

# 定义映射类LiveCells，其继承上一步创建的Base
class LiveCells(Base):
    # 指定本类映射到livecells表
    __tablename__ = 'livecells'
    
    id = Column(Integer, primary_key=True)
    tx_hash = Column(String(66))
    cell_index = Column(Integer)
    capacity = Column(Integer)
    lock_code_hash = Column(String(66))
    lock_args = Column(Text)
    lock_hash_type = Column(String(7))
    type_code_hash = Column(String(66))
    type_args = Column(Text)
    type_hash_type = Column(String(7))
    height = Column(Integer)

    def __repr__(self):
        return "<LiveCells(id = '%s', tx_hash='%s', cell_index='%s', capacity='%s', lock_code_hash='%s', lock_args='%s', lock_hash_type='%s', type_code_hash='%s', type_args='%s', type_hash_type='%s', height='%s')>" % (
                   self.id, self.tx_hash, self.cell_index, self.capacity, self.lock_code_hash, self.lock_args, self.lock_hash_type, self.type_code_hash, self.type_args, self.type_hash_type, self.height)

In [16]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
print(session)

In [17]:
sync_info = session.query(Info).first()
print(sync_info)
cell_info = session.query(LiveCells).filter_by(height=500).all()
print(cell_info)

2019-11-07 15:55:45,845 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-07 15:55:45,847 INFO sqlalchemy.engine.base.Engine ()
2019-11-07 15:55:45,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-07 15:55:45,852 INFO sqlalchemy.engine.base.Engine ()
2019-11-07 15:55:45,853 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-07 15:55:45,855 INFO sqlalchemy.engine.base.Engine SELECT info.tip_height AS info_tip_height 
FROM info
 LIMIT ? OFFSET ?
2019-11-07 15:55:45,856 INFO sqlalchemy.engine.base.Engine (1, 0)
<Info(tip_height='4293')>
2019-11-07 15:55:45,862 INFO sqlalchemy.engine.base.Engine SELECT livecells.id AS livecells_id, livecells.tx_hash AS livecells_tx_hash, livecells.cell_index AS livecells_cell_index, livecells.capacity AS livecells_capacity, livecells.lock_code_hash AS livecells_lock_code_hash, livecells.lock_args AS livecells_lock_args, livecells.lock_has

In [21]:
from pyDatalog import pyDatalog
def twice(a):
    return a+a

pyDatalog.create_terms('twice, X, Y')
print((X==2) & (Y==twice(X)))

X | Y
--|--
2 | 4


In [30]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from pyDatalog import pyDatalog
from sqlalchemy import Column, Integer, String, Text

engine = create_engine('sqlite:////home/rink/work/github/ckb-rpc-wrapper/sync.db?check_same_thread=False', echo=False)
# define a base class with SQLAlchemy and pyDatalog capabilities
Base = declarative_base(cls=pyDatalog.Mixin, metaclass=pyDatalog.sqlMetaMixin)
# open a session on a database, then associate it to the Base class
Session = sessionmaker(bind=engine)
session = Session()
Base.session = session
Base.metadata.bind = engine

class Info(Base):
    __tablename__ = 'info'
    __table_args__ = {'autoload':True} # autoload the model
    
    def __repr__(self):
        return "<Info(tip_height='%s')>" % (
                   self.tip_height)

class LiveCells(Base):
    __tablename__ = 'livecells'
    __table_args__ = {'autoload':True} # autoload the model
    
    def __repr__(self):
        return "<LiveCells(id = '%s', tx_hash='%s', cell_index='%s', capacity='%s', lock_code_hash='%s', lock_args='%s', lock_hash_type='%s', type_code_hash='%s', type_args='%s', type_hash_type='%s', height='%s')>" % (
                   self.id, self.tx_hash, self.cell_index, self.capacity, self.lock_code_hash, self.lock_args, self.lock_hash_type, self.type_code_hash, self.type_args, self.type_hash_type, self.height)

# which livecell's height is 500
X = pyDatalog.Variable()
LiveCells.height[X] == 500
print(X)
# which livecell's capacity > 203850000000
Y = pyDatalog.Variable()
LiveCells.capacity[Y] > 203850000000
print(Y)

[<LiveCells(id = '489', tx_hash='0x93b1d56938c20e26689bab69f46f45b522018c0f27a3d0fab9382060b98a5fe9', cell_index='0', capacity='203264251627', lock_code_hash='0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', lock_args='0x4a88cef22e4e71c48c40da51c1d6bd16daa97aa7', lock_hash_type='type', type_code_hash='', type_args='', type_hash_type='', height='500')>]
[<LiveCells(id = '1', tx_hash='0xa920249333f8816d51ce7fa09e949aeb290b8c54ed96de8efc629d4bb5612c47', cell_index='0', capacity='203850161678', lock_code_hash='0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', lock_args='0x4a88cef22e4e71c48c40da51c1d6bd16daa97aa7', lock_hash_type='type', type_code_hash='', type_args='', type_hash_type='', height='12')>]


In [31]:
!ls

demo.ipynb  README.md  vote.ipynb


In [32]:
!gcc -v

使用内建 specs。
COLLECT_GCC=gcc
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-pc-linux-gnu/9.2.0/lto-wrapper
目标：x86_64-pc-linux-gnu
配置为：/build/gcc/src/gcc/configure --prefix=/usr --libdir=/usr/lib --libexecdir=/usr/lib --mandir=/usr/share/man --infodir=/usr/share/info --with-bugurl=https://bugs.archlinux.org/ --enable-languages=c,c++,ada,fortran,go,lto,objc,obj-c++,d --enable-shared --enable-threads=posix --with-system-zlib --with-isl --enable-__cxa_atexit --disable-libunwind-exceptions --enable-clocale=gnu --disable-libstdcxx-pch --disable-libssp --enable-gnu-unique-object --enable-linker-build-id --enable-lto --enable-plugin --enable-install-libiberty --with-linker-hash-style=gnu --enable-gnu-indirect-function --enable-multilib --disable-werror --enable-checking=release --enable-default-pie --enable-default-ssp --enable-cet=auto gdc_include_dir=/usr/include/dlang/gdc
线程模型：posix
gcc 版本 9.2.0 (GCC) 
